In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.ndimage import gaussian_filter1d
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from datetime import datetime

In [ ]:
def get_params(df):
    port1_prob = df.loc[:, "rewprobfull1"].to_numpy()
    port2_prob = df.loc[:, "rewprobfull2"].to_numpy()
    port_choice = df.loc[:, "port"].to_numpy().astype(int)
    prob_choice = df.loc[:,"rw"].to_numpy()
    is_choice_high = ((port1_prob>=port2_prob)&(prob_choice>=port2_prob))|((port2_prob>port1_prob)&(prob_choice>port1_prob))
    is_reward = df.loc[:, "reward"].to_numpy()
    whr_reward_trial = np.where(is_reward == 1)[0]
    rwd_mov_avg = np.convolve(is_reward, np.ones(150) / 150, mode="same")
    # rwd_mov_avg_smth = gaussian_filter1d(rwd_mov_avg, 20)
    rwd_prob_corr = stats.pearsonr(port1_prob,port2_prob)

    choice_arr = np.vstack((port1_prob,port2_prob,prob_choice)).T

    return choice_arr,is_choice_high, is_reward, rwd_prob_corr

In [59]:
f = basepath/'gronckle.csv'
df = pd.read_csv(f)
arr,*_,is_choice_high = get_params(df)

In [60]:
datetime.fromtimestamp(1696509517)

datetime.datetime(2023, 10, 5, 18, 8, 37)

In [71]:
eptime = df['eptime'].to_numpy()
dt_time = pd.to_datetime(eptime,unit='s')

In [72]:
dt_time

DatetimeIndex(['2023-09-27 17:51:48', '2023-09-27 17:51:51',
               '2023-09-27 17:52:04', '2023-09-27 17:52:10',
               '2023-09-27 17:52:11', '2023-09-27 17:52:17',
               '2023-09-27 17:52:25', '2023-09-27 17:52:26',
               '2023-09-27 17:52:31', '2023-09-27 17:52:41',
               ...
               '2023-12-11 03:37:31', '2023-12-11 03:37:33',
               '2023-12-11 03:37:34', '2023-12-11 03:37:36',
               '2023-12-11 03:37:37', '2023-12-11 03:37:38',
               '2023-12-11 03:37:40', '2023-12-11 03:37:41',
               '2023-12-11 03:37:43', '2023-12-11 03:37:45'],
              dtype='datetime64[ns]', length=119127, freq=None)

In [ ]:
bad_choice = arr[np.where(~is_choice_high)]
bad_choice

array([[30, 70, 70],
       [30, 70, 70],
       [30, 70, 70],
       ...,
       [20, 80, 80],
       [20, 80, 80],
       [20, 80, 80]], shape=(107711, 3))

In [14]:
basepath = Path("D:\\Data")
files = ["gronckle.csv", "grump.csv"]
files = sorted(basepath.glob("*.csv"))[:2]

fig, axs = plt.subplots(2, 2, sharex="col")

for i, file in enumerate(files):
    data_df = pd.read_csv(basepath / file)
    rwd_prob1, rwd_prob2, port_choice, is_reward, rwd_prob_corr = get_params(data_df)

    # whr_reward_trial = np.where(is_reward == 1)[0]
    rwd_mov_avg = np.convolve(is_reward, np.ones(100) / 100, mode="same")
    rwd_mov_avg_smth = gaussian_filter1d(rwd_mov_avg, 20)

    rwd_prob_corr = stats.pearsonr(rwd_prob1, rwd_prob2)
    print(rwd_prob_corr)

    axs[0, i].plot(rwd_prob1, color="g")
    # axs[0].vlines(whr_reward_trial,10,20,alpha=0.1)
    axs[0, i].plot(rwd_prob2, color="r")
    axs[1, i].plot(rwd_mov_avg_smth, "k")
    axs[1, i].vlines(np.where(port_choice == 1), 0.1, 0.2, alpha=0.5, color="g")
    axs[1, i].vlines(np.where(port_choice == 2), 0.3, 0.4, alpha=0.5, color="r")

PearsonRResult(statistic=np.float64(-0.10355914090792981), pvalue=np.float64(6.964489685443776e-149))
PearsonRResult(statistic=np.float64(-0.10355914090792981), pvalue=np.float64(6.964489685443776e-149))


In [6]:
sorted(basepath.glob("*.csv"))[0]

WindowsPath('D:/Data/aggro.csv')

In [ ]:
data_df

,Unnamed: 0,trial#,trialstart,port,reward,trialend,session#,eptime,task,rewprobfull1,rewprobfull2,rw,animal,datetime
0,0,0,4092,2.0,1,4119,1,1696509517,13,40,70,70,Grump,2023-10-05 12:38:37
1,1,1,5721,2.0,0,5721,1,1696509519,13,40,70,70,Grump,2023-10-05 12:38:39
2,2,2,6790,2.0,1,6873,1,1696509520,13,40,70,70,Grump,2023-10-05 12:38:40
3,3,3,10714,2.0,1,10744,1,1696509524,13,40,70,70,Grump,2023-10-05 12:38:44
4,4,4,12221,2.0,1,12221,1,1696509526,13,40,70,70,Grump,2023-10-05 12:38:46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76104,76104,76104,1162679,2.0,1,1163022,167,1702264773,13,80,90,90,Grump,2023-12-11 03:19:33
76105,76105,76105,1164634,2.0,1,1164880,167,1702264775,13,80,90,90,Grump,2023-12-11 03:19:35
76106,76106,76106,1166300,2.0,1,1166421,167,1702264777,13,80,90,90,Grump,2023-12-11 03:19:37
76107,76107,76107,1167738,2.0,1,1167836,167,1702264779,13,80,90,90,Grump,2023-12-11 03:19:39


In [18]:
plt.plot(rwd_prob1,'.')

In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Simulate some synthetic data for 1000 trials
np.random.seed(42)
n_trials = 1000
n_history = 5  # Number of past trials considered (5 as in the equation)

# Simulate reward history (1 for reward, 0 for no reward)
reward_left = np.random.randint(
    0, 2, size=(n_trials, n_history)
)  # Left rewards history
reward_right = np.random.randint(
    0, 2, size=(n_trials, n_history)
)  # Right rewards history

# Simulate choice history (1 for chosen, 0 for unchosen)
choice_left = np.random.randint(
    0, 2, size=(n_trials, n_history)
)  # Left choices history
choice_right = np.random.randint(
    0, 2, size=(n_trials, n_history)
)  # Right choices history

# Simulate the actual choice (1 for left, 0 for right, as the target variable)
# The actual choice depends on some hidden logic; we'll just generate random choices for this example
actual_choice = np.random.randint(0, 2, size=n_trials)  # 1 for left, 0 for right

# Prepare the feature matrix (X) for the logistic regression model
# We combine reward and choice histories for both left and right
X = np.hstack(
    [
        reward_left - reward_right,  # reward history difference
        choice_left - choice_right,  # choice history difference
    ]
)

# Standardize the data (important for logistic regression)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, actual_choice, test_size=0.2, random_state=42
)

# Logistic Regression Model
log_reg = LogisticRegression(solver="lbfgs", max_iter=1000)
log_reg.fit(X_train, y_train)

# Print model coefficients (βr, βc, and βbias)
print("Model coefficients:")
print("Beta (Reward history weights):", log_reg.coef_[:, :n_history])
print("Beta (Choice history weights):", log_reg.coef_[:, n_history:])
print("Intercept (Bias term):", log_reg.intercept_)

# Predict on test data
y_pred = log_reg.predict(X_test)

# Evaluate model performance
accuracy = np.mean(y_pred == y_test)
print("Test accuracy:", accuracy)

Model coefficients:
Beta (Reward history weights): [[-0.03594947 -0.09515691  0.04508803 -0.08742038 -0.02923884]]
Beta (Choice history weights): [[ 0.0133873   0.0166049   0.07716208 -0.04840661 -0.0341045 ]]
Intercept (Bias term): [0.07619375]
Test accuracy: 0.515


In [6]:
actual_choice

array([0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,